In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import gc
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()
import warnings
from datetime import date, timedelta
warnings.filterwarnings('ignore')

In [2]:
sku_basic_info=pd.read_csv('data_b/jdata_sku_basic_info.csv')
user_basic_info=pd.read_csv('data_b/jdata_user_basic_info.csv')
user_action=pd.read_csv('data_b/jdata_user_action.csv')
user_order=pd.read_csv('data_b/jdata_user_order.csv')
user_comment_score=pd.read_csv('data_b/jdata_user_comment_score.csv')
print sku_basic_info.shape,user_basic_info.shape,user_action.shape,user_order.shape,user_comment_score.shape



(108333, 6) (99446, 4) (8033923, 5) (881692, 6) (233366, 4)


In [3]:
from SBBTree_ONLINE import SBBTree
%run com_util.py

In [75]:
#train label:201703
#val label:201704
#test  label:201705
#features:: order action



In [1]:
#o_date feature
#???




In [5]:
#order
#previous order


In [249]:
temp_user_order.shape

(100493, 18)

In [14]:
   

#target_month=201705
#start_month=201704
#feature_month=[start_month,target_month]

def get_order_month_feature(feature_month,name):
    print 'o_start'
    user_order=pd.read_csv('data_b/jdata_user_order.csv')
    user_order = user_order.merge(sku_basic_info,on=['sku_id'],how='left')
    user_order = user_order.merge(user_basic_info,on=['user_id'],how='left')
    user_order = user_order.merge(user_comment_score,on=['user_id','o_id'],how='left')
    # user_order['y_m_date'] = 100*pd.to_datetime(user_order['o_date']).dt.year+pd.to_datetime(user_order['o_date']).dt.month

    FLAG='O_'+str(name)+'_'
    temp_user_order=user_order[(user_order.o_date<=feature_month[1])&(user_order.o_date>=feature_month[0])]
    temp_user_order['comment_gap']=(pd.to_datetime(temp_user_order.comment_create_tm)-pd.to_datetime(temp_user_order.o_date)).dt.days.fillna(0)
    temp_user_order['o_date'] = pd.to_datetime(temp_user_order['o_date'])
    temp_user_order['order_day'] = temp_user_order['o_date'].dt.day
    temp_user_order['comment_create_tm'] = pd.to_datetime(temp_user_order['comment_create_tm'])
    temp_user_order['comment_day'] = temp_user_order['comment_create_tm'].dt.day
    temp_user_order['once_cost'] = temp_user_order.price*temp_user_order.o_sku_num
    #  
    sample = feat_count(user_basic_info[['user_id']],temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'sku_id',FLAG+'30101_')
    sample = feat_count(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'cate',FLAG+'no_30101_')
    sample = feat_count(sample,temp_user_order[(temp_user_order.cate==30)],['user_id'],'cate',FLAG+'30_')
    sample = feat_count(sample,temp_user_order[(temp_user_order.cate==101)],['user_id'],'cate',FLAG+'101_')
    
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate==30)],['user_id'],'once_cost',FLAG+'30_')
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate==101)],['user_id'],'once_cost',FLAG+'101_')
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'once_cost',FLAG+'30101_')
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'once_cost',FLAG+'no30101_')
    
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate==30)],['user_id'],'o_sku_num',FLAG+'30_')
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate==101)],['user_id'],'o_sku_num',FLAG+'101_')
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'o_sku_num',FLAG+'30101_')
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'o_sku_num',FLAG+'no30101_')

    sample = feat_count(sample,temp_user_order[((temp_user_order.cate==30))&(temp_user_order.comment_gap!=0)&(temp_user_order.score_level==1)],['user_id'],'score_level',FLAG+'1_no30_')
    sample = feat_count(sample,temp_user_order[((temp_user_order.cate==101))&(temp_user_order.comment_gap!=0)&(temp_user_order.score_level==1)],['user_id'],'score_level',FLAG+'1_no101_')
    sample = feat_count(sample,temp_user_order[((temp_user_order.cate!=101)&(temp_user_order.cate!=30))&(temp_user_order.comment_gap!=0)&(temp_user_order.score_level==1)],['user_id'],'score_level',FLAG+'1_no30101_')
    sample = feat_count(sample,temp_user_order[((temp_user_order.cate==101)|(temp_user_order.cate==30))&(temp_user_order.comment_gap!=0)&(temp_user_order.score_level==1)],['user_id'],'score_level',FLAG+'1_30101_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'comment_create_tm',FLAG+'no301010_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'comment_create_tm',FLAG+'30101_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==30)],['user_id'],'comment_create_tm',FLAG+'30_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==101)],['user_id'],'comment_create_tm',FLAG+'101_')
  #/////////////   mei you zhe xie xia jiang le a ? ?
    sample = feat_max(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'comment_day',FLAG+'no301010_')
    sample = feat_max(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'comment_day',FLAG+'30101_')

    sample = feat_min(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'order_day',FLAG+'no301010_')
    sample = feat_min(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'order_day',FLAG+'30101_')
    sample = feat_max(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'order_day',FLAG+'no301010_')
    sample = feat_max(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'order_day',FLAG+'30101_')
 #///////////////////you sha yong ? ?
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'o_date',FLAG+'no301010_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'o_date',FLAG+'30101_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==101)],['user_id'],'o_date',FLAG+'101_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==30)],['user_id'],'o_date',FLAG+'30_')
    temp_user_order_0 = temp_user_order.copy()
    temp_user_order_0 = merge_count(temp_user_order_0,['user_id'],'o_id','order_count')
    temp_user_order_0 = merge_max(temp_user_order_0,['user_id'],'o_date','order_max')
    temp_user_order_0 = merge_min(temp_user_order_0,['user_id'],'o_date','order_min')
    temp_user_order_0[FLAG+'feature_u1'] = (temp_user_order_0.order_max-temp_user_order_0.order_min).dt.days/(temp_user_order_0.order_count)
    temp_user_order_0 = temp_user_order_0[['user_id',FLAG+'feature_u1']]
    temp_user_order_0 = temp_user_order_0.drop_duplicates()
    sample = sample.merge(temp_user_order_0,on=['user_id'],how='left')
    sample=sample.drop(['user_id'],axis=1)
    print user_order.shape,temp_user_order.shape,sample.shape
    gc.collect()

    return sample,sample.columns.tolist()






In [ ]:


temp_user_order_0 = temp_user_order.copy()
temp_user_order_0 = merge_count(temp_user_order_0,['user_id'],'o_id','order_count')
temp_user_order_0 = merge_max(temp_user_order_0,['user_id'],'o_date','order_max')
temp_user_order_0 = merge_min(temp_user_order_0,['user_id'],'o_date','order_min')
temp_user_order_0[FLAG+'feature_u1'] = (temp_user_order_0.order_max-temp_user_order_0.order_min).dt.days/(temp_user_order_0.order_count)
temp_user_order_0 = temp_user_order_0[['user_id',FLAG+'feature_u1']]
temp_user_order_0 = temp_user_order_0.drop_duplicates()
sample = sample.merge(temp_user_order_0,on=['user_id'],how='left')



In [15]:
def get_order_one_month_feature(feature_month,name):
    print 'o_start'
    user_order=pd.read_csv('data_b/jdata_user_order.csv')
    user_order = user_order.merge(sku_basic_info,on=['sku_id'],how='left')
    user_order = user_order.merge(user_basic_info,on=['user_id'],how='left')
    user_order = user_order.merge(user_comment_score,on=['user_id','o_id'],how='left')
    # user_order['y_m_date'] = 100*pd.to_datetime(user_order['o_date']).dt.year+pd.to_datetime(user_order['o_date']).dt.month

    FLAG='O_'+str(name)+'_'
    temp_user_order=user_order[(user_order.o_date<=feature_month[1])&(user_order.o_date>=feature_month[0])]
    temp_user_order['comment_gap']=(pd.to_datetime(temp_user_order.comment_create_tm)-pd.to_datetime(temp_user_order.o_date)).dt.days.fillna(0)
    temp_user_order['o_date'] = pd.to_datetime(temp_user_order['o_date'])
    temp_user_order['order_day'] = temp_user_order['o_date'].dt.day    
    temp_user_order['comment_create_tm'] = pd.to_datetime(temp_user_order['comment_create_tm'])
    temp_user_order['comment_day'] = temp_user_order['comment_create_tm'].dt.day
    temp_user_order['once_cost'] = temp_user_order.price*temp_user_order.o_sku_num
    #  
    sample = feat_count(user_basic_info[['user_id']],temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'sku_id',FLAG+'30101_')
    sample = feat_count(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'cate',FLAG+'no_30101_')
    sample = feat_count(sample,temp_user_order[(temp_user_order.cate==30)],['user_id'],'cate',FLAG+'30_')
    sample = feat_count(sample,temp_user_order[(temp_user_order.cate==101)],['user_id'],'cate',FLAG+'101_')

   
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate==30)],['user_id'],'once_cost',FLAG+'30_')
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate==101)],['user_id'],'once_cost',FLAG+'101_')
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'once_cost',FLAG+'30101_')
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'once_cost',FLAG+'no30101_')

    sample = feat_count(sample,temp_user_order[((temp_user_order.cate==30))&(temp_user_order.comment_gap!=0)&(temp_user_order.score_level==1)],['user_id'],'score_level',FLAG+'1_no30_')
    sample = feat_count(sample,temp_user_order[((temp_user_order.cate==101))&(temp_user_order.comment_gap!=0)&(temp_user_order.score_level==1)],['user_id'],'score_level',FLAG+'1_no101_')
    sample = feat_count(sample,temp_user_order[((temp_user_order.cate!=101)&(temp_user_order.cate!=30))&(temp_user_order.comment_gap!=0)&(temp_user_order.score_level==1)],['user_id'],'score_level',FLAG+'1_no30101_')
    sample = feat_count(sample,temp_user_order[((temp_user_order.cate==101)|(temp_user_order.cate==30))&(temp_user_order.comment_gap!=0)&(temp_user_order.score_level==1)],['user_id'],'score_level',FLAG+'1_30101_')
    sample = feat_min(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'comment_day',FLAG+'no301010_')
    sample = feat_min(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'comment_day',FLAG+'30101_')
    sample = feat_max(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'comment_day',FLAG+'no301010_')
    sample = feat_max(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'comment_day',FLAG+'30101_')
  
    
    sample = feat_min(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'order_day',FLAG+'no301010_')
    sample = feat_min(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'order_day',FLAG+'30101_')
    sample = feat_max(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'order_day',FLAG+'no301010_')
    sample = feat_max(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'order_day',FLAG+'30101_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'order_day',FLAG+'no301010_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'order_day',FLAG+'30101_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==101)],['user_id'],'order_day',FLAG+'101_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==30)],['user_id'],'order_day',FLAG+'30_')
    temp_user_order_0 = temp_user_order.copy()
    temp_user_order_0 = merge_count(temp_user_order_0,['user_id'],'o_id','order_count')
    temp_user_order_0 = merge_max(temp_user_order_0,['user_id'],'o_date','order_max')
    temp_user_order_0 = merge_min(temp_user_order_0,['user_id'],'o_date','order_min')
    temp_user_order_0[FLAG+'feature_u1'] = (temp_user_order_0.order_max-temp_user_order_0.order_min).dt.days/(temp_user_order_0.order_count)
    temp_user_order_0 = temp_user_order_0[['user_id',FLAG+'feature_u1']]
    temp_user_order_0 = temp_user_order_0.drop_duplicates()
    sample = sample.merge(temp_user_order_0,on=['user_id'],how='left')
    sample[FLAG+'gap_end'] = pd.to_datetime(feature_month[1]).day-sample[FLAG+'30101_order_day_user_id_max']
    sample[FLAG+'gap_end_no'] = pd.to_datetime(feature_month[1]).day-sample[FLAG+'no301010_order_day_user_id_max']
    sample=sample.drop(['user_id'],axis=1)
    print user_order.shape,temp_user_order.shape,sample.shape
    gc.collect()

    return sample,sample.columns.tolist()

In [103]:
#feature 
user_order

,user_id,sku_id,o_id,o_date,o_area,o_sku_num
0,1,80036,480066,2017-03-09,14,1
1,1,16607,106933,2017-01-12,14,1
2,1,80036,542604,2017-04-14,14,1
3,1,80036,196335,2017-04-30,14,1
4,1,80036,42502,2017-03-22,14,2
5,1,16607,235938,2017-02-12,14,1
6,2,44666,554520,2017-04-24,27,2
7,3,71817,403068,2017-03-03,26,2
8,3,18217,403068,2017-03-03,26,2
9,3,2816,403068,2017-03-03,26,1


In [16]:
#feature 逐个月份是否购买　购买数目

# 聚焦S1
def get_order_every_month_feature(feature_month,name):
    print 'o_start'
    user_order=pd.read_csv('data_b/jdata_user_order.csv')
    user_order = user_order.merge(sku_basic_info,on=['sku_id'],how='left')
    user_order = user_order.merge(user_basic_info,on=['user_id'],how='left')
    user_order = user_order.merge(user_comment_score,on=['user_id','o_id'],how='left')
    # user_order['y_m_date'] = 100*pd.to_datetime(user_order['o_date']).dt.year+pd.to_datetime(user_order['o_date']).dt.month

    FLAG='Every_O_'+str(name)+'_'
    temp_user_order=user_order[(user_order.o_date<=feature_month[1])&(user_order.o_date>=feature_month[0])]
    temp_user_order['comment_gap']=(pd.to_datetime(temp_user_order.comment_create_tm)-pd.to_datetime(temp_user_order.o_date)).dt.days.fillna(0)
    temp_user_order['o_date'] = pd.to_datetime(temp_user_order['o_date'])
    temp_user_order['order_day'] = temp_user_order['o_date'].dt.day
    temp_user_order['comment_create_tm'] = pd.to_datetime(temp_user_order['comment_create_tm'])
    temp_user_order['comment_day'] = temp_user_order['comment_create_tm'].dt.day
    #  
    sample = feat_count(user_basic_info[['user_id']],temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'sku_id',FLAG+'30101_')
    sample = feat_count(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'cate',FLAG+'no_30101_')

    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'o_id',FLAG+'30101_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'o_id',FLAG+'no_30101_')

    sample = feat_mean(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'price',FLAG+'30101_')
    sample = feat_mean(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'price',FLAG+'no30101_')

    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'o_sku_num',FLAG+'30101_')
    sample = feat_sum(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'o_sku_num',FLAG+'no30101_')
    
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'comment_create_tm',FLAG+'no301010_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'comment_create_tm',FLAG+'30101_')
     
    
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate!=101)&(temp_user_order.cate!=30)],['user_id'],'o_date',FLAG+'no301010_')
    sample = feat_nunique(sample,temp_user_order[(temp_user_order.cate==101)|(temp_user_order.cate==30)],['user_id'],'o_date',FLAG+'30101_')
    sample=sample.drop(['user_id'],axis=1)
    print user_order.shape,temp_user_order.shape,sample.shape
    gc.collect()

    return sample,sample.columns.tolist()

In [24]:
user_action[user_action.user_id==1]

,user_id,sku_id,a_date,a_num,a_type
0,1,80036,2017-04-14,4,1
1,1,96959,2017-01-12,2,1
2,1,8017,2017-03-09,1,1
3,1,80036,2017-04-30,6,1
4,1,16607,2017-01-12,3,1
5,1,16607,2017-02-12,2,1
6,1,16607,2017-03-09,6,1
7,1,28038,2017-03-09,1,1
8,1,45314,2017-03-09,1,1
9,1,48100,2017-03-09,1,1


In [25]:
user_order[user_order.user_id==1]


,user_id,sku_id,o_id,o_date,o_area,o_sku_num
0,1,80036,480066,2017-03-09,14,1
1,1,16607,106933,2017-01-12,14,1
2,1,80036,542604,2017-04-14,14,1
3,1,80036,196335,2017-04-30,14,1
4,1,80036,42502,2017-03-22,14,2
5,1,16607,235938,2017-02-12,14,1


In [23]:
sample

,user_id,O_1_30101_sku_id_user_id_count,O_1_no_30101_cate_user_id_count,O_1_30101_o_id_user_id_nunique,O_1_no_30101_o_id_user_id_nunique,O_1_30101_price_user_id_mean,O_1_30101_o_sku_num_user_id_sum,O_1_no30101_o_sku_num_user_id_sum,O_1_1_no30101_score_level_user_id_count,O_1_1_30101_score_level_user_id_count,O_1_no301010_order_day_user_id_min,O_1_30101_order_day_user_id_min,O_1_no301010_order_day_user_id_max,O_1_30101_order_day_user_id_max,O_1_no301010_order_day_user_id_nunique,O_1_30101_order_day_user_id_nunique
0,1,2.0,NaN,2.0,NaN,511.800000,3.0,NaN,NaN,NaN,NaN,9.0,NaN,22.0,NaN,2.0
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,4.0,NaN,2.0,NaN,NaN,6.0,3.0,NaN,3.0,NaN,3.0,NaN,1.0,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,2.0,NaN,1.0,NaN,540.500000,2.0,NaN,NaN,NaN,NaN,25.0,NaN,25.0,NaN,1.0
7,8,1.0,NaN,1.0,NaN,401.600000,1.0,NaN,NaN,NaN,NaN,30.0,NaN,30.0,NaN,1.0
8,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,1.0,NaN,1.0,NaN,520.700000,1.0,NaN,NaN,NaN,NaN,16.0,NaN,16.0,NaN,1.0


In [4]:
#action
feature_month =['2017-03-01','2017-03-31']
name=1
FLAG='A_'+str(name)+'_'
user_action=pd.read_csv('data/jdata_user_action.csv')
user_action = user_action.merge(sku_basic_info,on=['sku_id'],how='left')
user_action = user_action.merge(user_basic_info,on=['user_id'],how='left')

#user_action['y_m_date'] = 100*pd.to_datetime(user_action['a_date']).dt.year+pd.to_datetime(user_action['a_date']).dt.month
temp_user_action=user_action[(user_action.a_date<=feature_month[1])&(user_action.a_date>=feature_month[0])]
temp_user_action['action_day'] = pd.to_datetime(temp_user_action['a_date']).dt.day
temp_user_action['action_month'] = pd.to_datetime(temp_user_action['a_date']).dt.month

sample = feat_nunique(user_basic_info[['user_id']],temp_user_action,['user_id'],'sku_id',FLAG)
sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate==101)|(temp_user_action.cate==30)],['user_id'],'sku_id',FLAG+'30101_')

sample = feat_nunique(sample,temp_user_action[temp_user_action.a_type==2],['user_id'],'sku_id',FLAG+'type2_')
sample = feat_nunique(sample,temp_user_action[temp_user_action.a_type==1],['user_id'],'sku_id',FLAG+'type1_')
sample = feat_sum(sample,temp_user_action[temp_user_action.a_type==1],['user_id'],'a_num',FLAG+'type1_')
sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==2)&((temp_user_action.cate==101)|(temp_user_action.cate==30))],['user_id'],'sku_id',FLAG+'type2_30101_')
sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate==101)|(temp_user_action.cate==30))],['user_id'],'sku_id',FLAG+'type1_30101')
sample = feat_sum(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate==101)|(temp_user_action.cate==30))],['user_id'],'a_num',FLAG+'type1_30101')

sample = feat_min(sample,temp_user_action[(temp_user_action.cate!=101)&(temp_user_action.cate!=30)],['user_id'],'action_day',FLAG+'no301010_')
sample = feat_min(sample,temp_user_action[(temp_user_action.cate==101)|(temp_user_action.cate==30)],['user_id'],'action_day',FLAG+'30101_')
sample = feat_max(sample,temp_user_action[(temp_user_action.cate!=101)&(temp_user_action.cate!=30)],['user_id'],'action_day',FLAG+'no301010_')
sample = feat_max(sample,temp_user_action[(temp_user_action.cate==101)|(temp_user_action.cate==30)],['user_id'],'action_day',FLAG+'30101_')
sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate!=101)&(temp_user_action.cate!=30)],['user_id'],'action_day',FLAG+'no301010_')
sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate==101)|(temp_user_action.cate==30)],['user_id'],'action_day',FLAG+'30101_')
sample[FLAG+'gap_end'] = pd.to_datetime(feature_month[1]).day-sample[FLAG+'30101_action_day_user_id_max']

In [17]:
#target_month=201705
#start_month=201704
#feature_month=[start_month,target_month]
def get_action_month_feature(feature_month,name):
    print 'action_start'
    FLAG='A_'+str(name)+'_'
    user_action=pd.read_csv('data_b/jdata_user_action.csv')
    user_action = user_action.merge(sku_basic_info,on=['sku_id'],how='left')
    user_action = user_action.merge(user_basic_info,on=['user_id'],how='left')

    #user_action['y_m_date'] = 100*pd.to_datetime(user_action['a_date']).dt.year+pd.to_datetime(user_action['a_date']).dt.month
    temp_user_action=user_action[(user_action.a_date<=feature_month[1])&(user_action.a_date>=feature_month[0])]


    sample = feat_nunique(user_basic_info[['user_id']],temp_user_action,['user_id'],'sku_id',FLAG)
    sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate==101)|(temp_user_action.cate==30)],['user_id'],'sku_id',FLAG+'30101_')
    sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate==101)],['user_id'],'sku_id',FLAG+'101_')
    sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate==30)],['user_id'],'sku_id',FLAG+'30_')

    sample = feat_nunique(sample,temp_user_action[temp_user_action.a_type==2],['user_id'],'sku_id',FLAG+'type2_')
    sample = feat_nunique(sample,temp_user_action[temp_user_action.a_type==1],['user_id'],'sku_id',FLAG+'type1_')
    sample = feat_sum(sample,temp_user_action[temp_user_action.a_type==1],['user_id'],'a_num',FLAG+'type1_')
    sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==2)&((temp_user_action.cate==30))],['user_id'],'sku_id',FLAG+'type2_30_')
    sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate==30))],['user_id'],'sku_id',FLAG+'type1_30')
    sample = feat_sum(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate==30))],['user_id'],'a_num',FLAG+'type1_30')
    sample = feat_sum(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate==101))],['user_id'],'a_num',FLAG+'type1_101')
    sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==2)&((temp_user_action.cate==101)|(temp_user_action.cate==30))],['user_id'],'sku_id',FLAG+'type2_30101_')
    sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate==101)|(temp_user_action.cate==30))],['user_id'],'sku_id',FLAG+'type1_30101')
    sample = feat_sum(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate==101)|(temp_user_action.cate==30))],['user_id'],'a_num',FLAG+'type1_30101')
    sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==2)&((temp_user_action.cate!=101)&(temp_user_action.cate!=30))],['user_id'],'sku_id',FLAG+'type2_no30101_')
    sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate!=101)&(temp_user_action.cate!=30))],['user_id'],'sku_id',FLAG+'type1_no30101')
    sample = feat_sum(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate!=101)&(temp_user_action.cate!=30))],['user_id'],'a_num',FLAG+'type1_no30101')

    
    
    sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate!=101)&(temp_user_action.cate!=30)],['user_id'],'a_date',FLAG+'no301010_')
    sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate==101)|(temp_user_action.cate==30)],['user_id'],'a_date',FLAG+'30101_')
    sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate==30)],['user_id'],'a_date',FLAG+'30_')
    sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate==101)],['user_id'],'a_date',FLAG+'101_')
    sample=sample.drop(['user_id'],axis=1)
    print user_action.shape,temp_user_action.shape,sample.shape
    gc.collect()

    return sample,sample.columns.tolist()


In [18]:
def get_action_one_month_feature(feature_month,name):
    print 'action_start'
    FLAG='A_'+str(name)+'_'
    user_action=pd.read_csv('data_b/jdata_user_action.csv')
    user_action = user_action.merge(sku_basic_info,on=['sku_id'],how='left')
    user_action = user_action.merge(user_basic_info,on=['user_id'],how='left')

    #user_action['y_m_date'] = 100*pd.to_datetime(user_action['a_date']).dt.year+pd.to_datetime(user_action['a_date']).dt.month
    temp_user_action=user_action[(user_action.a_date<=feature_month[1])&(user_action.a_date>=feature_month[0])]
    temp_user_action['action_day'] = pd.to_datetime(temp_user_action['a_date']).dt.day
    temp_user_action['action_month'] = pd.to_datetime(temp_user_action['a_date']).dt.month

    sample = feat_nunique(user_basic_info[['user_id']],temp_user_action,['user_id'],'sku_id',FLAG)
    sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate==101)|(temp_user_action.cate==30)],['user_id'],'sku_id',FLAG+'30101_')
    sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate==101)],['user_id'],'sku_id',FLAG+'101_')
    sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate==30)],['user_id'],'sku_id',FLAG+'30_')

    sample = feat_nunique(sample,temp_user_action[temp_user_action.a_type==2],['user_id'],'sku_id',FLAG+'type2_')
    sample = feat_nunique(sample,temp_user_action[temp_user_action.a_type==1],['user_id'],'sku_id',FLAG+'type1_')
    sample = feat_sum(sample,temp_user_action[temp_user_action.a_type==1],['user_id'],'a_num',FLAG+'type1_')
    sample = feat_sum(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate==30))],['user_id'],'a_num',FLAG+'type1_30')
    sample = feat_sum(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate==101))],['user_id'],'a_num',FLAG+'type1_101')
    sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==2)&((temp_user_action.cate==101)|(temp_user_action.cate==30))],['user_id'],'sku_id',FLAG+'type2_30101_')
    sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate==101)|(temp_user_action.cate==30))],['user_id'],'sku_id',FLAG+'type1_30101')
    sample = feat_sum(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate==101)|(temp_user_action.cate==30))],['user_id'],'a_num',FLAG+'type1_30101')
    sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==2)&((temp_user_action.cate!=101)&(temp_user_action.cate!=30))],['user_id'],'sku_id',FLAG+'type2_no30101_')
    sample = feat_count(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate!=101)&(temp_user_action.cate!=30))],['user_id'],'sku_id',FLAG+'type1_no30101')
    sample = feat_sum(sample,temp_user_action[(temp_user_action.a_type==1)&((temp_user_action.cate!=101)&(temp_user_action.cate!=30))],['user_id'],'a_num',FLAG+'type1_no30101')

    
    sample = feat_min(sample,temp_user_action[(temp_user_action.cate!=101)&(temp_user_action.cate!=30)],['user_id'],'action_day',FLAG+'no301010_')
    sample = feat_min(sample,temp_user_action[(temp_user_action.cate==101)|(temp_user_action.cate==30)],['user_id'],'action_day',FLAG+'30101_')
    sample = feat_max(sample,temp_user_action[(temp_user_action.cate!=101)&(temp_user_action.cate!=30)],['user_id'],'action_day',FLAG+'no301010_')
    sample = feat_max(sample,temp_user_action[(temp_user_action.cate==101)|(temp_user_action.cate==30)],['user_id'],'action_day',FLAG+'30101_')
    sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate!=101)&(temp_user_action.cate!=30)],['user_id'],'action_day',FLAG+'no301010_')
    sample = feat_nunique(sample,temp_user_action[(temp_user_action.cate==101)|(temp_user_action.cate==30)],['user_id'],'action_day',FLAG+'30101_')
    sample[FLAG+'gap_end'] = pd.to_datetime(feature_month[1]).day-sample[FLAG+'30101_action_day_user_id_max']
    sample=sample.drop(['user_id'],axis=1)
    print user_action.shape,temp_user_action.shape,sample.shape
    gc.collect()

    return sample,sample.columns.tolist()

In [9]:
#action order



In [19]:
#feature_month=[start , end]
def get_action_order_feature(feature_month):
    print 'a_o_start'
    user_action=pd.read_csv('data_b/jdata_user_action.csv')
    user_order=pd.read_csv('data_b/jdata_user_order.csv')
    user_action_order = pd.merge(user_action,user_order,on=['user_id','sku_id'],how='left')
    user_action_order = user_action_order.merge(sku_basic_info,on=['sku_id'],how='left')
    print user_action_order.shape
    user_action_order = user_action_order[((pd.to_datetime(user_action_order.a_date)<=pd.to_datetime(user_action_order.o_date.fillna('2018-01-01'))))]
    user_action_order = user_action_order.sort_values(by=['user_id','sku_id','a_date','o_date']).drop_duplicates(['user_id','sku_id','a_date'],keep="first")
    user_action_order = user_action_order[['user_id','sku_id','a_date','o_date','cate']]
    user_action_order['o_a_gap']=(pd.to_datetime(user_action_order.o_date)-pd.to_datetime(user_action_order.a_date)).dt.days 
    FLAG='a_o_'
    print user_action_order.shape
    user_action_order = user_action_order[((user_action_order.a_date>=feature_month[0])|(user_action_order.a_date<feature_month[1]))&((user_action_order.o_date>=feature_month[0])|(user_action_order.o_date<feature_month[1]))]
    print user_action_order.shape
    sample = feat_count(user_basic_info[['user_id']],user_action_order[user_action_order.o_a_gap>=-1],['user_id'],'sku_id',FLAG+'order_')
    sample = feat_count(sample,user_action_order,['user_id'],'sku_id',FLAG)
    sample['a_o_sku_id_rate']=(sample.a_o_order_sku_id_user_id_count+1)/(sample.a_o_sku_id_user_id_count+1)
    sample = feat_mean(sample,user_action_order[user_action_order.o_a_gap>=-1],['user_id'],'o_a_gap',FLAG)
    sample = feat_mean(sample,user_action_order[(user_action_order.o_a_gap>=-1)&((user_action_order.cate==101)|(user_action_order.cate==30))],['user_id'],'o_a_gap',FLAG+'30101_')

    sample=sample.drop(['user_id'],axis=1)
    print sample.shape
    gc.collect()

    return sample,sample.columns.tolist()

In [20]:
# train / val label
#target_month=201704

def get_label(target_month,month):
    #user_action=pd.read_csv('data/jdata_user_action.csv')
    user_order=pd.read_csv('data_b/jdata_user_order.csv')
    sku_basic_info=pd.read_csv('data_b/jdata_sku_basic_info.csv')
    user_basic_info=pd.read_csv('data_b/jdata_user_basic_info.csv')

    user_order = user_order.merge(sku_basic_info,on='sku_id',how='left')
    #user_action['y_m_date'] = 100*pd.to_datetime(user_action['a_date']).dt.year+pd.to_datetime(user_action['a_date']).dt.month
    #user_order['y_m_date'] = 100*pd.to_datetime(user_order['o_date']).dt.year+pd.to_datetime(user_order['o_date']).dt.month
    #user_action = user_action[['user_id','a_date']]
    user_order = user_order[['user_id','o_date','cate','o_id']]
    #user_action = user_action[user_action.y_m_date==(target_month-1)]
    #user_order = user_order[user_order.y_m_date==target_month]
    user_order = user_order[(user_order.o_date>=target_month[0])&(user_order.o_date<=target_month[1])]
    user_order = user_order[(user_order.cate==30)|(user_order.cate==101)]
    print user_order.shape
    
    sample = feat_nunique(user_basic_info[['user_id']],user_order,['user_id'],'o_id')
    sample.columns=['user_id','label_1']
    user_order = user_order.sort_values(by=['user_id','o_date']).drop_duplicates(["user_id"], keep="first")
   
    #user_action = user_action.sort_values(by=['user_id','a_date']).drop_duplicates(["user_id"], keep="last")
    label = pd.merge(user_basic_info[['user_id']],user_order[['user_id','o_date']],on=['user_id'],how='left')
    #label = label.merge(user_action[['user_id','a_date']],on=['user_id'],how='left')
    label = label.merge(sample,on=['user_id'],how='left')
    label['label_2'] = (pd.to_datetime(label["o_date"])-pd.to_datetime(date(2017,month,1))).dt.days.fillna(0)
    print label.shape
    return label


In [21]:
#train feature 201703
user_basic_info=pd.read_csv('data_b/jdata_user_basic_info.csv')

train_order_1,_ = get_order_one_month_feature(['2017-07-01','2017-07-31'],1)
train_order_2,_ = get_order_month_feature(['2017-05-01','2017-07-31'],2) 
train_order_3,_ = get_order_month_feature(['2017-02-01','2017-07-31'],3) 
train_order_4,_ = get_order_month_feature(['2016-11-01','2017-07-31'],4) 

#train_order_4,_ = get_order_every_month_feature(['2017-02-01','2017-02-28'],2)
#train_order_5,_ = get_order_every_month_feature(['2017-02-01','2017-03-31'],12)
#train_order_6,_ = get_order_every_month_feature(['2017-01-01','2017-01-31'],3)

train_action_1,_ = get_action_one_month_feature(['2017-07-01','2017-07-31'],1)
train_action_2,_ = get_action_month_feature(['2017-05-01','2017-07-31'],2) 
train_action_3,_ = get_action_month_feature(['2017-02-01','2017-07-31'],3) 
train_action_4,_ = get_action_month_feature(['2016-11-01','2017-07-31'],4) 
#train_action_order_0,_ = get_action_order_feature(['2016-11-01','2017-03-31'])
train_feature = pd.concat([user_basic_info[['age','sex','user_lv_cd']],train_order_1,train_order_2,train_order_3,
                         #  train_order_5,
                           train_action_1,train_action_2,train_action_3,train_order_4,train_action_4],axis=1)
print train_feature.shape
# train label
train_label = get_label(['2017-08-01','2017-08-30'],8)

train_feature_label1 = pd.concat([train_feature,train_label],axis=1)

 
#val feature 201704
val_order_1,_ = get_order_one_month_feature(['2017-06-01','2017-06-30'],1)
val_order_2,_ = get_order_month_feature(['2017-04-01','2017-06-30'],2)
val_order_3,_ = get_order_month_feature(['2017-01-01','2017-06-30'],3)
val_order_4,_ = get_order_month_feature(['2016-10-01','2017-06-30'],4)

#val_order_4,_ = get_order_every_month_feature(['2017-01-01','2017-01-31'],2)
#val_order_5,_ = get_order_every_month_feature(['2017-01-01','2017-02-28'],12)
#val_order_6,_ = get_order_every_month_feature(['2016-12-01','2016-12-30'],3)

val_action_1,_ = get_action_one_month_feature(['2017-06-01','2017-06-30'],1)
val_action_2,_ = get_action_month_feature(['2017-04-01','2017-06-30'],2)
val_action_3,_ = get_action_month_feature(['2017-01-01','2017-06-30'],3)
val_action_4,_ = get_action_month_feature(['2016-10-01','2017-06-30'],4)
#val_action_order_0,_ = get_action_order_feature(['2016-10-01','2017-02-28'])
val_feature = pd.concat([user_basic_info[['age','sex','user_lv_cd']],val_order_1,val_order_2,val_order_3,
                      #   val_order_5,
                         val_action_1,val_action_2,val_action_3,val_order_4,val_action_4],axis=1)
# val label
val_label = get_label(['2017-07-01','2017-07-31'],7)
print val_feature.shape

val_feature_label1 = pd.concat([val_feature,val_label],axis=1)
 
#test feature 201708
test_order_1,_ = get_order_one_month_feature(['2017-08-01','2017-08-31'],1)
test_order_2,_ = get_order_month_feature(['2017-06-01','2017-08-31'],2)
test_order_3,_ = get_order_month_feature(['2017-03-01','2017-08-31'],3)
test_order_4,_ = get_order_month_feature(['2016-12-01','2017-08-31'],4)

#test_order_4,_ = get_order_every_month_feature(['2017-03-01','2017-03-31'],2)
#test_order_5,_ = get_order_every_month_feature(['2017-03-01','2017-04-30'],12)
#test_order_6,_ = get_order_every_month_feature(['2017-02-01','2017-02-28'],3)

test_action_1,_ = get_action_one_month_feature(['2017-08-01','2017-08-31'],1)
test_action_2,_ = get_action_month_feature(['2017-06-01','2017-08-31'],2)
test_action_3,_ = get_action_month_feature(['2017-03-01','2017-08-31'],3)
test_action_4,_ = get_action_month_feature(['2016-12-01','2017-08-31'],4)
#test_action_order_0,_ = get_action_order_feature(['2016-12-01','2017-05-01'])
test_feature1 = pd.concat([user_basic_info[['age','sex','user_lv_cd']],test_order_1,test_order_2,test_order_3,
                        #   test_order_5,
                           test_action_1,test_action_3,test_action_2,test_order_4,test_action_4],axis=1)
print test_feature1.shape
 

o_start
(993571, 16) (101327, 20) (99446, 34)
o_start
(993571, 16) (364330, 20) (99446, 34)
o_start
(993571, 16) (598915, 20) (99446, 34)
o_start
(993571, 16) (789351, 20) (99446, 34)
action_start
(8033923, 13) (793544, 15) (99446, 28)
action_start
(8033923, 13) (2781988, 13) (99446, 23)
action_start
(8033923, 13) (4733748, 13) (99446, 23)
action_start
(8033923, 13) (6343178, 13) (99446, 23)
(99446, 236)
(63977, 4)
(99446, 4)
o_start
(993571, 16) (172033, 20) (99446, 34)
o_start
(993571, 16) (352082, 20) (99446, 34)
o_start
(993571, 16) (557825, 20) (99446, 34)
o_start
(993571, 16) (739871, 20) (99446, 34)
action_start
(8033923, 13) (1256200, 15) (99446, 28)
action_start
(8033923, 13) (2700968, 13) (99446, 23)
action_start
(8033923, 13) (4427963, 13) (99446, 23)
action_start
(8033923, 13) (6011650, 13) (99446, 23)
(60735, 4)
(99446, 4)
(99446, 236)
o_start
(993571, 16) (109204, 20) (99446, 34)
o_start
(993571, 16) (382564, 20) (99446, 34)
o_start
(993571, 16) (643509, 20) (99446, 34)
o

In [22]:
def label_change(data):
    if(data>=1):
        return data
    else:
        return 0
train_feature_label1['label_1'] = train_feature_label1.label_1.apply(label_change)
val_feature_label1['label_1'] = val_feature_label1.label_1.apply(label_change)

In [23]:
train_feature_label1

,age,sex,user_lv_cd,O_1_30101_sku_id_user_id_count,O_1_no_30101_cate_user_id_count,O_1_30_cate_user_id_count,O_1_101_cate_user_id_count,O_1_30_o_id_user_id_nunique,O_1_101_o_id_user_id_nunique,O_1_30101_o_id_user_id_nunique,...,A_4_type1_no30101sku_id_user_id_count,A_4_type1_no30101a_num_user_id_sum,A_4_no301010_a_date_user_id_nunique,A_4_30101_a_date_user_id_nunique,A_4_30_a_date_user_id_nunique,A_4_101_a_date_user_id_nunique,user_id,o_date,label_1,label_2
0,3,1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,74.0,171.0,25.0,19.0,7.0,18.0,1,NaN,0.0,0.0
1,3,0,3,2.0,0.0,0.0,2.0,0.0,1.0,1.0,...,12.0,19.0,6.0,25.0,3.0,25.0,2,NaN,0.0,0.0
2,4,2,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,30.0,34.0,12.0,12.0,7.0,8.0,3,NaN,0.0,0.0
3,3,2,4,1.0,0.0,0.0,1.0,0.0,1.0,1.0,...,10.0,11.0,1.0,21.0,2.0,21.0,4,2017-08-04,3.0,3.0
4,3,0,5,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,2.0,4.0,2.0,17.0,7.0,16.0,5,2017-08-09,3.0,8.0
5,2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6,NaN,0.0,0.0
6,3,1,4,2.0,0.0,0.0,2.0,0.0,2.0,2.0,...,6.0,11.0,5.0,13.0,0.0,13.0,7,NaN,0.0,0.0
7,-1,2,2,1.0,0.0,0.0,1.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,8,NaN,0.0,0.0
8,4,0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,2.0,1.0,13.0,5.0,10.0,9,NaN,0.0,0.0
9,3,0,4,0.0,7.0,0.0,0.0,0.0,0.0,0.0,...,62.0,160.0,22.0,4.0,2.0,4.0,10,NaN,0.0,0.0


In [ ]:
# model predict

In [24]:
params_1 = {
          'boosting_type': 'gbdt',
          'objective': 'regression_l2',
          'metric': 'auc',
          'min_child_weight': 1.5,
          'num_leaves': 2**5,
          'lambda_l2': 10,
          'subsample': 0.7,
          'colsample_bytree': 0.5,
          'colsample_bylevel': 0.5,
          'learning_rate': 0.1,
          'scale_pos_weight': 20,
          'seed': 2018,
          'bagging_fraction': 0.8,
          'bagging_freq': 5,
          'nthread': 4,
          'silent': True,
          }



In [25]:
feature_list = test_feature1.columns.tolist()
# train_feature_label, val_feature_label
# test_feature
print len(feature_list)

236


In [80]:
#SBBTree
model = SBBTree(params=params_1, stacking_num=5, bagging_num=3,  bagging_test_size=0.33, 
                num_boost_round=10000, early_stopping_rounds=100)
X = train_feature_label1[feature_list].values
y = train_feature_label1['label_1'].values
X_pred = test_feature1[feature_list].values

In [81]:
model.fit(X,y)

[1]	valid_0's auc: 0.78268
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.786515
[3]	valid_0's auc: 0.787411
[4]	valid_0's auc: 0.791085
[5]	valid_0's auc: 0.791969
[6]	valid_0's auc: 0.794007
[7]	valid_0's auc: 0.793835
[8]	valid_0's auc: 0.794862
[9]	valid_0's auc: 0.795559
[10]	valid_0's auc: 0.796294
[11]	valid_0's auc: 0.796391
[12]	valid_0's auc: 0.796684
[13]	valid_0's auc: 0.797135
[14]	valid_0's auc: 0.797897
[15]	valid_0's auc: 0.798356
[16]	valid_0's auc: 0.798151
[17]	valid_0's auc: 0.798163
[18]	valid_0's auc: 0.798583
[19]	valid_0's auc: 0.79906
[20]	valid_0's auc: 0.799191
[21]	valid_0's auc: 0.799287
[22]	valid_0's auc: 0.799653
[23]	valid_0's auc: 0.799643
[24]	valid_0's auc: 0.799797
[25]	valid_0's auc: 0.799965
[26]	valid_0's auc: 0.80036
[27]	valid_0's auc: 0.800729
[28]	valid_0's auc: 0.800941
[29]	valid_0's auc: 0.800878
[30]	valid_0's auc: 0.800863
[31]	valid_0's auc: 0.801039
[32]	valid_0's auc: 0.801334
[33]	valid_0's auc: 0

[280]	valid_0's auc: 0.803849
[281]	valid_0's auc: 0.803918
[282]	valid_0's auc: 0.803959
[283]	valid_0's auc: 0.803853
[284]	valid_0's auc: 0.803904
[285]	valid_0's auc: 0.803839
[286]	valid_0's auc: 0.803809
[287]	valid_0's auc: 0.803804
[288]	valid_0's auc: 0.803813
[289]	valid_0's auc: 0.803861
[290]	valid_0's auc: 0.803942
[291]	valid_0's auc: 0.803931
[292]	valid_0's auc: 0.803933
[293]	valid_0's auc: 0.803961
[294]	valid_0's auc: 0.803994
[295]	valid_0's auc: 0.803962
[296]	valid_0's auc: 0.803874
[297]	valid_0's auc: 0.803865
[298]	valid_0's auc: 0.80379
[299]	valid_0's auc: 0.803797
[300]	valid_0's auc: 0.803868
[301]	valid_0's auc: 0.803904
[302]	valid_0's auc: 0.803942
[303]	valid_0's auc: 0.80397
[304]	valid_0's auc: 0.803994
[305]	valid_0's auc: 0.803981
[306]	valid_0's auc: 0.803988
[307]	valid_0's auc: 0.804072
[308]	valid_0's auc: 0.804064
[309]	valid_0's auc: 0.804021
[310]	valid_0's auc: 0.804097
[311]	valid_0's auc: 0.804139
[312]	valid_0's auc: 0.804183
[313]	valid_

[242]	valid_0's auc: 0.804365
[243]	valid_0's auc: 0.804511
[244]	valid_0's auc: 0.804511
[245]	valid_0's auc: 0.804564
[246]	valid_0's auc: 0.804568
[247]	valid_0's auc: 0.804562
[248]	valid_0's auc: 0.804568
[249]	valid_0's auc: 0.804444
[250]	valid_0's auc: 0.804466
[251]	valid_0's auc: 0.804466
[252]	valid_0's auc: 0.804384
[253]	valid_0's auc: 0.804321
[254]	valid_0's auc: 0.804251
[255]	valid_0's auc: 0.804289
[256]	valid_0's auc: 0.804298
[257]	valid_0's auc: 0.8043
[258]	valid_0's auc: 0.804377
[259]	valid_0's auc: 0.804373
[260]	valid_0's auc: 0.804377
[261]	valid_0's auc: 0.804364
[262]	valid_0's auc: 0.804373
[263]	valid_0's auc: 0.804404
[264]	valid_0's auc: 0.804596
[265]	valid_0's auc: 0.804598
[266]	valid_0's auc: 0.804639
[267]	valid_0's auc: 0.804598
[268]	valid_0's auc: 0.804565
[269]	valid_0's auc: 0.804585
[270]	valid_0's auc: 0.804612
[271]	valid_0's auc: 0.804615
[272]	valid_0's auc: 0.804589
[273]	valid_0's auc: 0.804497
[274]	valid_0's auc: 0.804529
[275]	valid_

[219]	valid_0's auc: 0.814679
[220]	valid_0's auc: 0.814719
[221]	valid_0's auc: 0.814626
[222]	valid_0's auc: 0.814576
[223]	valid_0's auc: 0.814672
[224]	valid_0's auc: 0.814584
[225]	valid_0's auc: 0.814609
[226]	valid_0's auc: 0.814526
[227]	valid_0's auc: 0.814537
[228]	valid_0's auc: 0.81445
[229]	valid_0's auc: 0.814427
[230]	valid_0's auc: 0.814369
[231]	valid_0's auc: 0.814431
[232]	valid_0's auc: 0.814532
[233]	valid_0's auc: 0.814498
[234]	valid_0's auc: 0.814547
[235]	valid_0's auc: 0.814515
[236]	valid_0's auc: 0.814551
[237]	valid_0's auc: 0.814561
[238]	valid_0's auc: 0.814574
[239]	valid_0's auc: 0.814599
[240]	valid_0's auc: 0.814515
[241]	valid_0's auc: 0.814483
[242]	valid_0's auc: 0.814549
[243]	valid_0's auc: 0.81454
[244]	valid_0's auc: 0.814508
[245]	valid_0's auc: 0.814575
[246]	valid_0's auc: 0.8146
[247]	valid_0's auc: 0.814678
[248]	valid_0's auc: 0.814721
[249]	valid_0's auc: 0.814691
[250]	valid_0's auc: 0.81462
[251]	valid_0's auc: 0.814638
[252]	valid_0's

[192]	valid_0's auc: 0.80957
[193]	valid_0's auc: 0.809558
[194]	valid_0's auc: 0.809506
[195]	valid_0's auc: 0.809543
[196]	valid_0's auc: 0.809506
[197]	valid_0's auc: 0.809508
[198]	valid_0's auc: 0.809519
[199]	valid_0's auc: 0.809505
Early stopping, best iteration is:
[99]	valid_0's auc: 0.810362
[1]	valid_0's auc: 0.78276
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.788211
[3]	valid_0's auc: 0.790176
[4]	valid_0's auc: 0.793633
[5]	valid_0's auc: 0.794466
[6]	valid_0's auc: 0.79652
[7]	valid_0's auc: 0.797514
[8]	valid_0's auc: 0.797798
[9]	valid_0's auc: 0.798624
[10]	valid_0's auc: 0.799159
[11]	valid_0's auc: 0.800265
[12]	valid_0's auc: 0.800465
[13]	valid_0's auc: 0.800761
[14]	valid_0's auc: 0.800946
[15]	valid_0's auc: 0.801207
[16]	valid_0's auc: 0.801554
[17]	valid_0's auc: 0.801711
[18]	valid_0's auc: 0.801813
[19]	valid_0's auc: 0.802525
[20]	valid_0's auc: 0.80281
[21]	valid_0's auc: 0.803026
[22]	valid_0's auc: 0.803243
[23]	val

[86]	valid_0's auc: 0.804034
[87]	valid_0's auc: 0.804089
[88]	valid_0's auc: 0.80406
[89]	valid_0's auc: 0.804059
[90]	valid_0's auc: 0.80406
[91]	valid_0's auc: 0.804079
[92]	valid_0's auc: 0.804064
[93]	valid_0's auc: 0.804051
[94]	valid_0's auc: 0.80409
[95]	valid_0's auc: 0.804129
[96]	valid_0's auc: 0.804139
[97]	valid_0's auc: 0.804061
[98]	valid_0's auc: 0.803982
[99]	valid_0's auc: 0.803968
[100]	valid_0's auc: 0.803939
[101]	valid_0's auc: 0.80389
[102]	valid_0's auc: 0.803921
[103]	valid_0's auc: 0.803914
[104]	valid_0's auc: 0.803948
[105]	valid_0's auc: 0.803918
[106]	valid_0's auc: 0.803881
[107]	valid_0's auc: 0.803807
[108]	valid_0's auc: 0.803825
[109]	valid_0's auc: 0.803797
[110]	valid_0's auc: 0.80376
[111]	valid_0's auc: 0.803732
[112]	valid_0's auc: 0.803738
[113]	valid_0's auc: 0.80373
[114]	valid_0's auc: 0.803751
[115]	valid_0's auc: 0.80373
[116]	valid_0's auc: 0.803712
[117]	valid_0's auc: 0.80372
[118]	valid_0's auc: 0.803707
[119]	valid_0's auc: 0.803739
[1

[141]	valid_0's auc: 0.806478
[142]	valid_0's auc: 0.80644
[143]	valid_0's auc: 0.806362
[144]	valid_0's auc: 0.806308
[145]	valid_0's auc: 0.806362
[146]	valid_0's auc: 0.806309
[147]	valid_0's auc: 0.806272
[148]	valid_0's auc: 0.806256
[149]	valid_0's auc: 0.806204
[150]	valid_0's auc: 0.806188
[151]	valid_0's auc: 0.806169
[152]	valid_0's auc: 0.806149
[153]	valid_0's auc: 0.806124
[154]	valid_0's auc: 0.806122
[155]	valid_0's auc: 0.806096
[156]	valid_0's auc: 0.806049
[157]	valid_0's auc: 0.806086
[158]	valid_0's auc: 0.806032
[159]	valid_0's auc: 0.806063
[160]	valid_0's auc: 0.806058
[161]	valid_0's auc: 0.805989
[162]	valid_0's auc: 0.806008
[163]	valid_0's auc: 0.805988
[164]	valid_0's auc: 0.805963
[165]	valid_0's auc: 0.805933
[166]	valid_0's auc: 0.805878
[167]	valid_0's auc: 0.805808
[168]	valid_0's auc: 0.805821
[169]	valid_0's auc: 0.805754
[170]	valid_0's auc: 0.805728
[171]	valid_0's auc: 0.805662
[172]	valid_0's auc: 0.805596
[173]	valid_0's auc: 0.805554
[174]	valid

[166]	valid_0's auc: 0.803751
[167]	valid_0's auc: 0.803772
[168]	valid_0's auc: 0.803799
[169]	valid_0's auc: 0.803784
[170]	valid_0's auc: 0.80378
[171]	valid_0's auc: 0.803733
[172]	valid_0's auc: 0.803715
[173]	valid_0's auc: 0.803695
[174]	valid_0's auc: 0.803679
[175]	valid_0's auc: 0.803723
[176]	valid_0's auc: 0.803736
[177]	valid_0's auc: 0.803683
[178]	valid_0's auc: 0.803663
[179]	valid_0's auc: 0.803633
[180]	valid_0's auc: 0.803597
[181]	valid_0's auc: 0.803546
[182]	valid_0's auc: 0.803511
[183]	valid_0's auc: 0.803575
[184]	valid_0's auc: 0.803588
[185]	valid_0's auc: 0.803533
[186]	valid_0's auc: 0.803537
[187]	valid_0's auc: 0.803515
[188]	valid_0's auc: 0.803482
[189]	valid_0's auc: 0.803486
[190]	valid_0's auc: 0.803606
[191]	valid_0's auc: 0.803616
[192]	valid_0's auc: 0.803604
[193]	valid_0's auc: 0.803583
[194]	valid_0's auc: 0.803495
[195]	valid_0's auc: 0.803403
[196]	valid_0's auc: 0.803389
[197]	valid_0's auc: 0.803366
[198]	valid_0's auc: 0.803354
[199]	valid

In [97]:
test_tree = user_basic_info[['user_id']]
test_tree['prob'] = model.predict(X_pred)

In [98]:
test_tree = test_tree.sort_values("prob",ascending=False)

In [99]:
test_tree = test_tree.merge(test_y[['user_id','pred_date_0']],on=['user_id'],how='left')
test_tree=test_tree[:50000][["user_id","pred_date_0"]]
test_tree.columns=['user_id','pred_date']

#test_tree.to_csv("sub_new_S2_04_sbbtree.csv",index=None)

In [90]:
test_tree.sort_values("prob",ascending=False).to_csv("SBBTree_S1.csv",index=None)

In [82]:
test_cv

,index,user_id,prob0,prob1,prob2,pred_date,prob
0,13226,13227,80.567534,7.168270,82.805942,2017-09-06,56.847249
1,11996,11997,47.434154,6.178465,49.948133,2017-09-06,34.520251
2,22523,22524,35.497384,13.503022,34.972373,2017-09-02,27.990926
3,91701,91702,32.599834,6.639005,28.413328,2017-09-07,22.550722
4,84463,84464,21.854423,12.179396,20.295644,2017-09-06,18.109821
5,72328,72329,22.390054,7.217538,20.026501,2017-09-05,16.544697
6,72636,72637,22.313019,9.938792,15.949477,2017-09-12,16.067096
7,63291,63292,19.450849,4.017106,17.899584,2017-09-11,13.789179
8,23118,23119,19.143620,4.364982,15.808248,2017-09-07,13.105616
9,8111,8112,14.970661,10.770194,11.469269,2017-09-04,12.403375


In [57]:
X_train.shape,X_test.shape

((69246, 196), (29678, 196))

In [53]:
from sklearn.model_selection import train_test_split
test_cv = test_feature1.copy()

for bn in range(3):
    X_train, X_test, y_train, y_test = train_test_split(train_feature_label1[feature_list], train_feature_label1['label_1'], test_size=0.3, random_state=bn)
    train_matrix=lgb.Dataset(X_train,label=y_train)
    valid_matrix=lgb.Dataset(X_test,label=y_test)
    model = lgb.train(params_1, train_matrix, num_boost_round=4000, valid_sets=valid_matrix,
                      early_stopping_rounds=100 )
    test_cv["prob"+str(bn)] = model.predict(test_cv[feature_list], num_iteration=model.best_iteration).reshape((test_cv.shape[0], 1))
test_cv = pd.concat([user_basic_info[['user_id']],test_cv[['prob0','prob1','prob2']]],axis=1)
pred_date = pd.read_csv('/home/hou/Windy/JDATA/data_b/S2_pred_date.csv')
test_cv = test_cv.merge(pred_date,on=['user_id'],how='left')
test_cv['prob'] = (test_cv.prob0+test_cv.prob1+test_cv.prob2)/3
test_cv = test_cv.sort_values("prob",ascending=False)

[1]	valid_0's auc: 0.781176
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.785594
[3]	valid_0's auc: 0.786734
[4]	valid_0's auc: 0.789711
[5]	valid_0's auc: 0.791281
[6]	valid_0's auc: 0.793006
[7]	valid_0's auc: 0.793411
[8]	valid_0's auc: 0.794271
[9]	valid_0's auc: 0.795011
[10]	valid_0's auc: 0.795359
[11]	valid_0's auc: 0.795896
[12]	valid_0's auc: 0.796304
[13]	valid_0's auc: 0.79635
[14]	valid_0's auc: 0.796898
[15]	valid_0's auc: 0.79722
[16]	valid_0's auc: 0.797493
[17]	valid_0's auc: 0.797416
[18]	valid_0's auc: 0.797837
[19]	valid_0's auc: 0.798246
[20]	valid_0's auc: 0.798427
[21]	valid_0's auc: 0.798366
[22]	valid_0's auc: 0.798544
[23]	valid_0's auc: 0.798805
[24]	valid_0's auc: 0.799074
[25]	valid_0's auc: 0.799346
[26]	valid_0's auc: 0.799505
[27]	valid_0's auc: 0.800002
[28]	valid_0's auc: 0.800267
[29]	valid_0's auc: 0.800456
[30]	valid_0's auc: 0.800428
[31]	valid_0's auc: 0.800599
[32]	valid_0's auc: 0.800869
[33]	valid_0's auc: 

[75]	valid_0's auc: 0.80708
[76]	valid_0's auc: 0.807074
[77]	valid_0's auc: 0.807115
[78]	valid_0's auc: 0.807204
[79]	valid_0's auc: 0.807189
[80]	valid_0's auc: 0.807153
[81]	valid_0's auc: 0.807104
[82]	valid_0's auc: 0.807109
[83]	valid_0's auc: 0.807125
[84]	valid_0's auc: 0.807169
[85]	valid_0's auc: 0.807165
[86]	valid_0's auc: 0.807114
[87]	valid_0's auc: 0.807063
[88]	valid_0's auc: 0.806994
[89]	valid_0's auc: 0.806989
[90]	valid_0's auc: 0.806921
[91]	valid_0's auc: 0.80692
[92]	valid_0's auc: 0.806916
[93]	valid_0's auc: 0.806941
[94]	valid_0's auc: 0.806889
[95]	valid_0's auc: 0.80694
[96]	valid_0's auc: 0.806952
[97]	valid_0's auc: 0.806882
[98]	valid_0's auc: 0.806911
[99]	valid_0's auc: 0.806842
[100]	valid_0's auc: 0.806838
[101]	valid_0's auc: 0.806862
[102]	valid_0's auc: 0.806918
[103]	valid_0's auc: 0.806974
[104]	valid_0's auc: 0.807024
[105]	valid_0's auc: 0.806959
[106]	valid_0's auc: 0.806971
[107]	valid_0's auc: 0.806969
[108]	valid_0's auc: 0.806898
[109]	va

[81]	valid_0's auc: 0.804001
[82]	valid_0's auc: 0.804133
[83]	valid_0's auc: 0.80409
[84]	valid_0's auc: 0.804037
[85]	valid_0's auc: 0.804054
[86]	valid_0's auc: 0.804099
[87]	valid_0's auc: 0.80409
[88]	valid_0's auc: 0.804081
[89]	valid_0's auc: 0.804137
[90]	valid_0's auc: 0.804215
[91]	valid_0's auc: 0.804225
[92]	valid_0's auc: 0.804134
[93]	valid_0's auc: 0.804194
[94]	valid_0's auc: 0.804182
[95]	valid_0's auc: 0.804256
[96]	valid_0's auc: 0.804272
[97]	valid_0's auc: 0.804271
[98]	valid_0's auc: 0.804211
[99]	valid_0's auc: 0.804299
[100]	valid_0's auc: 0.804321
[101]	valid_0's auc: 0.804309
[102]	valid_0's auc: 0.804271
[103]	valid_0's auc: 0.804234
[104]	valid_0's auc: 0.804297
[105]	valid_0's auc: 0.804273
[106]	valid_0's auc: 0.804386
[107]	valid_0's auc: 0.804547
[108]	valid_0's auc: 0.804451
[109]	valid_0's auc: 0.804554
[110]	valid_0's auc: 0.804496
[111]	valid_0's auc: 0.804489
[112]	valid_0's auc: 0.804362
[113]	valid_0's auc: 0.804396
[114]	valid_0's auc: 0.804396
[

In [ ]:
## 109 features 819316 816034 0.815107
## 139 0.841548 0.837859 0.836691
## 133 0.842316 0.838348 0.836546

In [31]:
test_cv = test_cv.merge(test_y[['user_id','pred_date_0']],on=['user_id'],how='left')
test_cv_0=test_cv[:50000][["user_id","pred_date_0"]]
test_cv_0.columns=['user_id','pred_date']

test_cv_0.to_csv("sub_new_S2_02.csv",index=None)

In [30]:
params_2 = {
          'boosting_type': 'gbdt',
          'objective': 'regression_l2',
          'metric': 'mse',
          'min_child_weight': 1.5,
          'num_leaves': 2**5,
          'lambda_l2': 10,
          'subsample': 0.7,
          'colsample_bytree': 0.5,
          'colsample_bylevel': 0.5,
          'learning_rate': 0.1,
          'scale_pos_weight': 20,
          'seed': 2018,
          'nthread': 4,
          'silent': True,
          }

In [31]:
##S2
train_x = train_feature_label1[train_feature_label1.label_1>=1][feature_list].values
train_y = train_feature_label1[train_feature_label1.label_1>=1]['label_2'].values

val_x = val_feature_label1[val_feature_label1.label_1>=1][feature_list].values
val_y = val_feature_label1[val_feature_label1.label_1>=1]['label_2'].values

train_matrix=lgb.Dataset(train_x,label=train_y)
valid_matrix=lgb.Dataset(val_x,label=val_y)
model = lgb.train(params_2, train_matrix, num_boost_round=600, valid_sets=valid_matrix,
                  early_stopping_rounds=100
                  )


[1]	valid_0's l2: 76.2532
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's l2: 75.8794
[3]	valid_0's l2: 75.6261
[4]	valid_0's l2: 75.4154
[5]	valid_0's l2: 75.2519
[6]	valid_0's l2: 75.0389
[7]	valid_0's l2: 74.8608
[8]	valid_0's l2: 74.7371
[9]	valid_0's l2: 74.567
[10]	valid_0's l2: 74.4208
[11]	valid_0's l2: 74.3242
[12]	valid_0's l2: 74.2367
[13]	valid_0's l2: 74.1851
[14]	valid_0's l2: 74.1007
[15]	valid_0's l2: 74.0356
[16]	valid_0's l2: 74.0206
[17]	valid_0's l2: 73.9958
[18]	valid_0's l2: 73.9511
[19]	valid_0's l2: 73.9229
[20]	valid_0's l2: 73.8808
[21]	valid_0's l2: 73.8586
[22]	valid_0's l2: 73.8474
[23]	valid_0's l2: 73.8115
[24]	valid_0's l2: 73.7994
[25]	valid_0's l2: 73.7669
[26]	valid_0's l2: 73.7424
[27]	valid_0's l2: 73.6892
[28]	valid_0's l2: 73.6767
[29]	valid_0's l2: 73.6925
[30]	valid_0's l2: 73.6832
[31]	valid_0's l2: 73.6689
[32]	valid_0's l2: 73.6483
[33]	valid_0's l2: 73.6333
[34]	valid_0's l2: 73.6396
[35]	valid_0's l2: 73.6451
[3

In [32]:
test_y = user_basic_info[['user_id']]

test_x = test_feature1.copy()
test_y["gap"] = model.predict(test_x[feature_list], num_iteration=model.best_iteration).reshape((test_x.shape[0], 1))
test_y['pred_date_0'] = test_y['gap'].map(lambda day: datetime(2017, 9, 1)+timedelta(days=int(day+0.49-1)))


In [34]:
test_y[['user_id','pred_date_0']].to_csv("data_b/S2_pred_date_2.csv",index=None)

In [28]:
test_cv = test_cv[['user_id','prob']]
test_cv = test_cv.merge(test_y,on=['user_id'],how='left')

In [18]:

test_cv[:50000][["user_id","pred_date"]].to_csv("sub_new_S2_01.csv",index=None)

In [75]:
valid_matrix=lgb.Dataset(train_x,label=train_y)
train_matrix=lgb.Dataset(val_x,label=val_y)

In [190]:
##S1
train_x = train_feature_label1[feature_list].values
train_y = train_feature_label1['label_1'].values

#val_x = val_feature_label1[feature_list].values
#val_y = val_feature_label1['label_1'].values

train_matrix=lgb.Dataset(train_x,label=train_y)
#valid_matrix=lgb.Dataset(val_x,label=val_y)
model = lgb.train(params_1, train_matrix, num_boost_round=900,
                 , valid_sets=valid_matrix,early_stopping_rounds=100
                  )

[1]	training's auc: 0.821479
[2]	training's auc: 0.829047
[3]	training's auc: 0.830838
[4]	training's auc: 0.832063
[5]	training's auc: 0.833407
[6]	training's auc: 0.834299
[7]	training's auc: 0.835495
[8]	training's auc: 0.835873
[9]	training's auc: 0.836165
[10]	training's auc: 0.836244
[11]	training's auc: 0.837018
[12]	training's auc: 0.837187
[13]	training's auc: 0.837782
[14]	training's auc: 0.837857
[15]	training's auc: 0.838146
[16]	training's auc: 0.838283
[17]	training's auc: 0.838371
[18]	training's auc: 0.838498
[19]	training's auc: 0.838381
[20]	training's auc: 0.838599
[21]	training's auc: 0.838732
[22]	training's auc: 0.839054
[23]	training's auc: 0.839268
[24]	training's auc: 0.839355
[25]	training's auc: 0.83965
[26]	training's auc: 0.839809
[27]	training's auc: 0.839942
[28]	training's auc: 0.840019
[29]	training's auc: 0.840198
[30]	training's auc: 0.840429
[31]	training's auc: 0.840623
[32]	training's auc: 0.840847
[33]	training's auc: 0.84102
[34]	training's auc: 

In [191]:
feat_imp = pd.Series(model.feature_importance(), feature_list).sort_values(ascending=False)
feat_imp

O_12_30101_o_sku_num_user_id_sum                   102
O_3_30101_o_sku_num_user_id_sum                     95
O_2_30101_o_sku_num_user_id_sum                     79
O_1_30101_price_user_id_mean                        76
O_1_30101_o_sku_num_user_id_sum                     74
O_12_no_30101_o_id_user_id_nunique                  68
O_1_30101_order_day_user_id_max                     62
O_3_30101_o_date_user_id_nunique                    62
user_lv_cd                                          61
O_3_no_30101_o_id_user_id_nunique                   55
A_1_30101_action_day_user_id_max                    55
O_1_30101_order_day_user_id_min                     53
O_12_30101_o_id_user_id_nunique                     52
A_1_30101_action_day_user_id_nunique                51
O_3_30101_price_user_id_mean                        50
O_3_30101_o_id_user_id_nunique                      48
O_2_30101_o_date_user_id_nunique                    46
A_1_gap_end                                         44
A_3_type1_

In [105]:
#S1 score
import math
def label_change_back(data):
    if(data>=1):
        return 1
    else:
        return 0
val_feature_label1['label_1'] = val_feature_label1.label_1.apply(label_change_back)
val_pred = pd.concat([user_basic_info[['user_id']],val_feature_label1[['label_1']]],axis=1)
val_pred['pred'] = model.predict(val_feature_label1[feature_list]).reshape((val_pred.shape[0], 1))
val_pred = val_pred.sort_values("pred",ascending=False).reset_index()
val_pred['index_']=val_pred.index+1
def get_w(data):
    return (1.0/(1.0+math.log(data)))
val_pred['s1_w'] = val_pred.index_.apply(get_w)
val_pred['s1_fenzi'] = val_pred.label_1*val_pred.s1_w
(val_pred[['s1_fenzi']][:50000].sum().values/val_pred[['s1_w']][:50000].sum().values)[0]

0.506639226880124

In [ ]:
0.586824777264406

In [64]:
test_x = test_feature1.copy()
test_x["prob"] = model.predict(test_x[feature_list], num_iteration=model.best_iteration).reshape((test_x.shape[0], 1))
test_y = pd.concat([user_basic_info[['user_id']],test_x[['prob']]],axis=1)
pred_date = pd.read_csv('/home/hou/Windy/JDATA/S2_pred_date.csv')
test_y = test_y.merge(pred_date,on=['user_id'],how='left')
test_y = test_y.sort_values("prob",ascending=False).reset_index()


In [83]:
test_y.describe()

,index,user_id,prob
count,98924.000000,98924.000000,98924.000000
mean,49461.500000,49462.500000,0.571587
std,28557.043352,28557.043352,0.542514
min,0.000000,1.000000,-5.280102
25%,24730.750000,24731.750000,0.317309
50%,49461.500000,49462.500000,0.478470
75%,74192.250000,74193.250000,0.709076
max,98923.000000,98924.000000,42.688533


In [87]:
test_y

,index,user_id,prob,pred_date
0,34499,34500,42.688533,2017-05-07
1,56297,56298,42.321274,2017-05-09
2,6912,6913,33.353475,2017-05-09
3,75124,75125,30.968653,2017-05-08
4,12188,12189,26.339100,2017-05-07
5,65048,65049,20.666604,2017-05-08
6,53265,53266,20.272033,2017-05-07
7,75997,75998,18.448224,2017-05-07
8,39386,39387,17.908024,2017-05-09
9,37361,37362,17.146712,2017-05-08


index             78049
user_id           78050
prob           0.474891
pred_date    2017-05-15
Name: 50000, dtype: object

In [32]:
test_cv_0

,user_id,pred_date
0,13227,2017-09-06
1,11997,2017-09-06
2,22524,2017-09-07
3,91702,2017-09-08
4,84464,2017-09-06
5,72329,2017-09-06
6,72637,2017-09-12
7,63292,2017-09-09
8,23119,2017-09-06
9,8112,2017-09-07


In [69]:
# test_x = test_feature[feature_list]
# test_x["day_gap"] = model.predict(test_x, num_iteration=model.best_iteration).reshape((test_x.shape[0], 1))
# test_label = pd.concat([test_label,test_x[['day_gap']]],axis=1)
# def pred_date(x,y):
#     a=int(x[:4])
#     b=int(x[5:7])
#     c=int(x[8:10])
#     return str(date(a,b,c)+timedelta(round(y,0)))
# test_label["pred_date"]=list(map(lambda x,y:pred_date(x,y),test_label["a_date"],test_label["day_gap"]))
# def pre_label_1(pred_date):
#     if (pred_date>'2017-05-01')&(pred_date<'2017-05-25'):
#         return 1
#     else:
#         return 0
# test_label['pre_label_1'] = test_label.pred_date.apply(pre_label_1)

In [86]:
test_y[:50000][["user_id","pred_date"]].to_csv("sub_bad_kuri_3.csv",index=None)

In [37]:
test_cv = test_cv[['user_id','pred_date','pred_date_0']]
test_cv['day_low'] = pd.to_datetime(test_cv['pred_date']).dt.day
test_cv['day_high'] = pd.to_datetime(test_cv['pred_date_0']).dt.day

In [4]:
pred_1=pd.read_csv("S2_pred_date.csv")
pred_2=pd.read_csv("data_b/S2_pred_date.csv")

In [7]:
pred_1['day'] = pd.to_datetime(pred_1['pred_date']).dt.day
pred_2['day'] = pd.to_datetime(pred_2['pred_date']).dt.day

In [102]:
test_tree['day'] = pd.to_datetime(test_tree['pred_date']).dt.day

In [41]:
test_cv[:50000].describe()

,user_id,day_low,day_high
count,50000.000000,50000.000000,50000.000000
mean,49775.287040,12.977560,12.851960
std,28668.167188,2.059646,2.112376
min,3.000000,2.000000,5.000000
25%,24987.750000,12.000000,12.000000
50%,49805.500000,13.000000,13.000000
75%,74657.250000,14.000000,14.000000
max,99446.000000,22.000000,22.000000


In [39]:
val_feature_label1[val_feature_label1.label_1>=1]["label_2"].describe()


count    40168.000000
mean        14.454441
std          8.679808
min          0.000000
25%          7.000000
50%         14.000000
75%         21.000000
max         30.000000
Name: label_2, dtype: float64

In [103]:
test_tree.describe()

,user_id,day
count,50000.000000,50000.000000
mean,49771.896860,12.859600
std,28658.464943,2.118541
min,3.000000,5.000000
25%,24955.750000,12.000000
50%,49839.500000,13.000000
75%,74598.250000,14.000000
max,99446.000000,22.000000


In [106]:
def change_date(data):
    if data>=8:
        return data-1
    else:
        return data
def change_date_2(data):
    return data/1.1
test_S2_B = pd.read_csv("sub_new_S2_02.csv")
test_S2_B['day'] = pd.to_datetime(test_S2_B['pred_date']).dt.day
test_S2_B['date_new'] = test_S2_B.day.apply(change_date)
test_S2_B['pred_date_0'] = test_S2_B['date_new'].map(lambda day: datetime(2017, 9, 1)+timedelta(days=int(day+0.49-1)))


In [109]:
test_S2_B = test_S2_B[['user_id','pred_date_0']]
test_S2_B.columns = ['user_id','pred_date']
test_S2_B.to_csv("sub_new_S2_date_change.csv",index=None)